# 贪心算法

## 定义
贪心算法（又称贪婪算法）是指，在对问题求解时，总是做出在当前看来是最好的选择。也就是说，不从整体最优上加以考虑，他所做出的是在某种意义上的局部最优解。

## 特点
贪婪算法可解决的问题通常大部分都有如下的特性：  
1. 随着算法的进行，将积累起其它两个集合：一个包含已经被考虑过并被选出的候选对象，另一个包含已经被考虑过但被丢弃的候选对象。
2. 有一个函数来检查一个候选对象的集合是否提供了问题的解答。该函数不考虑此时的解决方法是否最优。
3. 还有一个函数检查是否一个候选对象的集合是可行的，也即是否可能往该集合上添加更多的候选对象以获得一个解。和上一个函数一样，此时不考虑解决方法的最优性。
4. 选择函数可以指出哪一个剩余的候选对象最有希望构成问题的解。
5. 最后，目标函数给出完整的解。

## 过程

1. 建立数学模型来描述问题；
2. 把求解的问题分成若干个子问题；
3. 对每一子问题求解，得到子问题的局部最优解；
4. 把子问题的解局部最优解合成原来解问题的一个解。

## 例子
建立数据集满足以下条件:
1. 平均年龄在35岁到45岁之间
2. 工资总额在10万到12万之间

## 解决方案
求平均年龄和总工资的MSE最小值。

In [5]:
import pandas as pd
import random, time
import numpy as np

## 建立随机数据集
数据集包含以下列:
1. 性别: 随机生成“男性”或“女性”  
2. 年龄: 22岁至65岁之间的随机整数。
3. 工资: 3000 - 10000之间的随机整数

In [6]:
n_row = 1000
random.seed(50)

# create a series of gender
gender = pd.Series([random.choice(['male','female']) for i in range(n_row)])

# create a series of age
age_low = 22
age_high = 65
age = pd.Series([random.randint(age_low, age_high) for i in range(n_row)])

# create a series of salary
salary_low = 3000
salary_high = 10000
salary = pd.Series([random.randint(salary_low, salary_high) for i in range(n_row)])

# create a dataframe by gender and salary
df = pd.DataFrame({"gender": gender,"age": age, "salary": salary})
df.head()

,age,gender,salary
0,64,female,3828
1,64,female,6389
2,39,female,8344
3,30,male,4540
4,33,female,5147


## 创建一个字符串字典及其相应的函数。
1. "average": numpy.mean function  
2. "sum": numpy.sum function

In [7]:
def str2func(x):
    func_dict = {"average": np.mean, "sum": np.sum}
    return func_dict[x]

## 计算平方误差
假设我们希望误差x落在[a, b]区间上，那么平方误差的计算如下: 
1. if x in [a, b], then $SE = 0$
2. if x > b, then $SE = (x / b - 1)^2$
3. if x < a，then $SE = (1 - x / a)^2$


In [8]:
def get_se(x, rng):
    a, b = rng
    if a <= x <= b:
        res = 0
    elif x > b:
        # Normalization
        res = (x / b - 1) ** 2
    else:
        res = (1 - x / a) ** 2
    return res

## 计算均方误差
$MSE = \frac{1}{n}\sum_{i}^{n}SE_i$

In [9]:
def get_mse(data, rows, cols, funcs, rngs, n_cond):
    mse = 0.0
    for col, func, rng in zip(cols, funcs, rngs):
        se = func(data.loc[rows == 1, col])
        se = get_se(se, rng)   
        mse += se / n_cond
    return mse

## 搜索功能
变量行类似于[1,1,0,1,0,0,0…]，其中1表示这一行被选中。将mse和min_mse设置为“正无穷”作为哨兵使代码更加优雅。

1. 创建一个带有n个零的数组
2. 计算未被选中数据的mse
3. 在步骤2中记录最小的mse为min_mse，并将相应的数据标号设置为1
4. 比较mse和min_mse，然后更新mse的值
5. 如果mse不能再降低，就终止迭代

In [10]:
def search(data, cols, funcs, rngs, threshold=10e-6):
    n_row = data.shape[0]
    n_cond = len(cols)
    
    # create a series to show which rows are selected
    rows = pd.Series(np.zeros(n_row, dtype = np.int32))
    rows.index = data.index
    
    # get functions
    funcs = [str2func(x) for x in funcs]

    i = 0
    mse = float('inf')
    while mse > threshold:
        min_mse = float('inf')
        for idx in data.loc[rows == 0].index:
            rows.loc[idx] = 1
            tmp_mse = get_mse(data, rows, cols, funcs, rngs, n_cond)
            
            if tmp_mse < min_mse:
                min_mse = tmp_mse
                min_mse_idx = idx
            else:
                pass
            
            rows.loc[idx] = 0
        
        # check if mse cannot be lower any more
        if min_mse > mse:
            break
        else:
            mse = min_mse
            rows.loc[min_mse_idx] = 1
        
        # print loss
        print("%d times iteration, mse %.3f" % (i+1, mse))
        i += 1
        
    return rows

## 测试

In [11]:
print("\n" * 3)
print("Test search:")

run_time = time.time()

idxs = search(data = df
              , cols = ["age", "salary"]
              , funcs = ["average", "sum"]
              , rngs = [[35,40], [100000, 120000]])

search_result = df.loc[idxs == 1]
average_age = search_result.age.mean()
total_salary = search_result.salary.sum()

print()
print("Target average age is 35 to 40 and target total salary is 100000 to 120000")
print("Average age is %.2f and total salary is %d" % (average_age, total_salary))
print("Run time is %.2f s" % (time.time() - run_time))





Test search:
1 times iteration, mse 0.405
2 times iteration, mse 0.321
3 times iteration, mse 0.246
4 times iteration, mse 0.181
5 times iteration, mse 0.126
6 times iteration, mse 0.081
7 times iteration, mse 0.046
8 times iteration, mse 0.021
9 times iteration, mse 0.005
10 times iteration, mse 0.000
11 times iteration, mse 0.000

Target average age is 35 to 40 and target total salary is 100000 to 120000
Average age is 39.00 and total salary is 103287
Run time is 10.84 s


In [12]:
search_result

,age,gender,salary
0,64,female,3828
176,22,male,9945
222,39,female,9980
415,59,female,9921
449,27,female,9986
497,24,female,9944
602,43,male,9935
725,36,male,9937
846,60,female,9975
878,22,male,9922
